In [5]:
!pip install flask
!pip install numpy
!pip install opencv-contrib-python
!pip install imutils
!pip install python-ffmpeg-video-streaming

  Created wheel for imutils: filename=imutils-0.5.3-py3-none-any.whl size=25850 sha256=b131feba8a24eb3380e0d8ef57b9c238205ea13579ecdd4dc6aaa479ddf5da95
  Stored in directory: /home/oem/.cache/pip/wheels/fc/9c/6d/1826267c72afa51b564c9c6e0f66abc806879338bc593a2270
Successfully built imutils


In [6]:
import numpy as np
import imutils
import cv2

class SingleMotionDetector:
    def __init__(self, accumWeight=0.5):
        self.accumWeight = accumWeight
        self.bg = None
        
    def update(self, img):
        if self.bg is None:
            self.bg = img.copy().astype("float")
            return
        
        cv2.accumulateWeighted(img, self.bg, self.accumWeight)
        
    def detect(self, img, tVal = 25):
        delta = cv2.absdiff(self.bg.astype("uint8"), img)
        thresh = cv2.threshold(delta, tVal, 255, cv2.THRESH_BINARY)[1]
        
        thresh = cv2.erode(thresh, None, iterations=2)
        thresh = cv2.dilate(thresh, None, iterations=2)
        
        cnts = cv2.findContours(thresh.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        cnts = imutils.grab_contours(cnt)
        (minX, minY) = (np.inf, np.inf)
        (maxX, maxY) = (-np.inf, -np.inf)
        
        if len(cnts) == 0:
            return None
        
        for c in cnts:
            (x, y, w, h) = cv2.boundingRect(c)
            (minX, minY) = (min(minX, x), min(minY, y))
            (maxX, maxY) = (max(maxX, x + w), max(maxY, y + h))
            
        return (thresh, (minX, minY, maxX, maxY))

In [7]:
from imutils.video import VideoStream
from flask import Response
from flask import Flask
from flask import render_template

import threading
import argparse
import datetime
import imutils
import time
import cv2

In [11]:
outputFrame = None
lock = threading.Lock()

app = Flask(__name__)

vs = VideoStream(src = 0).start()
time.sleep(2.0)

@app.route("/")
def index():
    return render_template("index.html")

def detect_motion(frameCount):
    global vs, outputFrame, lock
    md = SingleMotionDetector(accumWeight = 0.1)
    total = 0
    
    while True:
        frame = vs.read()
        frame = imutils.resize(frame, width = 400)
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        gray = cv2.GaussianBlur(gray, (7, 7), 0)
        
        timestampe = datetime.datetime.now()
        cv2.putText(frame, timestamp.strftime("%A %d %B %Y %I: %M: %S%p"),
                   (10, frame.shape[0] - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.35, (0, 0, 255), 1)
        
        if total > frameCount:
            motion = md.detect(gray)
            
            if motion is not None:
                (thresh, (minX, minY, maxX, maxY)) = motion
                cv2.rectangle(frame, (minX, minY), (maxX, maxY), (0, 0, 255), 2)
                
        md.update(gray)
        total += 1
        
        with lock:
            outputFrame = frame.copy()
            
def generate():
    global outputFrame, lock
    
    while True:
        with lock:
            if outputFrame is None:
                continue
                
            (flag, encodedImage) = cv2.imencode(".jpg", outputFrame)
            
            if not flag:
                continue
                
        yield(b'--frame\r\n' b'Content-Type: image/jpeg\r\n\r\n' + bytearray(encodedImage) + b'\r\n')
        
@app.route('/video_feed')
def video_feed():
    return Response(generate(), mimetype = "multipart/x-mixed-replace; boundary=frame")

In [ ]:
if __name__ == '__main__':
    #ap = argparse.ArgumentParser()
    #ap.add_argument("-i", "--ip", type=str, required=True, help="ip address of the device")
    #ap.add_argument("-o", "--port", type=int, required=True,
    #                help="ephemeral port number of the server (1024 to 65535)")
    #ap.add_argument("-f", "--frame-count", type=int, default=32,
    #                help="# of frames used to construct the background model")
    #args = vars(ap.parse_args())

    t = threading.Thread(target=detect_motion, args=(32,))
    t.daemon = True
    t.start()

    #app.run(host=args["ip"], port=args["port"], debug=True, threaded=True, use_reloader=False)
    app.run(host="0.0.0.0", port=33333, debug=True, threaded=True, use_reloader=False)

vs.stop()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on


Exception in thread Thread-6:
Traceback (most recent call last):
  File "/home/oem/anaconda3/lib/python3.7/threading.py", line 926, in _bootstrap_inner
    self.run()
  File "/home/oem/anaconda3/lib/python3.7/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-11-82c26b24eef0>", line 20, in detect_motion
    frame = imutils.resize(frame, width = 400)
  File "/home/oem/anaconda3/lib/python3.7/site-packages/imutils/convenience.py", line 69, in resize
    (h, w) = image.shape[:2]
AttributeError: 'NoneType' object has no attribute 'shape'

INFO:werkzeug: * Running on http://0.0.0.0:33333/ (Press CTRL+C to quit)


In [ ]:
import ffmpeg_streaming

In [4]:
#video = ffmpeg_streaming.input('/var/media/video.mp4')
video = ffmpeg_streaming.input('https://www.')